In [1]:
from sklearn.metrics import f1_score
import numpy as np

In [2]:
# Borrowed from http://vict0rsch.github.io/2018/06/06/tensorflow-streaming-multilabel-f1/
def alter_data(_data):
    data = _data.copy()
    new_data = []
    for d in data:
        for i, l in enumerate(d):
            if np.random.rand() < 0.2:
                d[i] = (d[i] + 1) % 2
        new_data.append(d)
    return np.array(new_data)

def get_data():
    # Number of different classes
    num_classes = 10
    classes = list(range(num_classes))
    # Numberof samples in synthetic dataset
    examples = 10000
    # Max number of labels per sample. Minimum is 1
    max_labels = 5
    class_probabilities = np.array(
        list(6 * np.exp(-i * 5 / num_classes) + 1 for i in range(num_classes))
    )
    class_probabilities /= class_probabilities.sum()
    labels = [
        np.random.choice(
            classes, # Choose labels in 0..num_classes
            size=np.random.randint(1, max_labels), # number of labels for this sample
            p=class_probabilities,  # Probability of drawing each class
            replace=False,  # A class can only occure once
        )
        for _ in range(examples)  # Do it `examples` times
    ]
    y_true = np.zeros((examples, num_classes)).astype(np.int64)
    for i, l in enumerate(labels):
        y_true[i][l] = 1
    y_pred = alter_data(y_true)
    return y_true, y_pred

In [3]:
np.random.seed(0)
y_true, y_pred = get_data()
num_classes = y_true.shape[-1]

In [4]:
# Current Implementation in FastAI library (modified to accept numpy arrays)
def fbeta(y_pred, y_true, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True) :
    "Computes the f_beta between `y_pred` and `y_true` in a multi-classification task."
    beta2 = beta**2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh)#.float()
    y_true = y_true#.float()
    TP = (y_pred*y_true).sum(axis=1)
    prec = TP/(y_pred.sum(axis=1)+eps)
    rec = TP/(y_true.sum(axis=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean()

In [5]:
# Modified fbeta with axis = 0
def fbetaMod(y_pred, y_true, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True) :
    "Computes the f_beta between `y_pred` and `y_true` in a multi-classification task."
    beta2 = beta**2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh)#.float()
    y_true = y_true#.float()
    TP = (y_pred*y_true).sum(axis=0)
    prec = TP/(y_pred.sum(axis=0)+eps)
    rec = TP/(y_true.sum(axis=0)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean()

In [6]:
f1_current = fbeta(y_pred, y_true, beta=1, thresh=0, sigmoid=False)
f1_modified = fbetaMod(y_pred, y_true, beta=1, thresh=0, sigmoid=False)
print("{:40}".format("\nFastAI f1 scores: (current, modified)"), f1_current, f1_modified)


FastAI f1 scores: (current, modified)   0.6374086212650519 0.6241802913905903


In [7]:
mic = f1_score(y_true, y_pred, average="micro")
mac = f1_score(y_true, y_pred, average="macro")
wei = f1_score(y_true, y_pred, average="weighted")
print("{:40}".format("\nFor reference, scikit f1 scores: (micro, macro, weighted)"), mic, mac, wei)


For reference, scikit f1 scores: (micro, macro, weighted) 0.665699032365699 0.6241802918567531 0.6868241897597981
